In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image

In [5]:
class bicubic(nn.Module):
    def __init__(self):
        super(bicubic, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')

    def forward(self, x):
        x = self.interpolation(x)
        return x


In [11]:
# Load the trained model
class FSRCNN(nn.Module):
    def __init__(self, d=100, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=4, padding=4, output_padding=3)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [15]:

model = bicubic()
model.load_state_dict(torch.load("/kaggle/input/bicubic/bicubic.pth"))
model.eval()

# Preprocess the test image
path = "/kaggle/working/dog_low_res/"
filename = "downsampled_flickr_dog_000243.jpg"
test_image_path = path+filename
test_image = Image.open(test_image_path)
transform = transforms.Compose([
    transforms.ToTensor()
])
test_image = transform(test_image).unsqueeze(0)

# Apply the model to the test image
with torch.no_grad():
    output_image = model(test_image)

# Convert the output tensor to an image
output_image = output_image.squeeze(0)
output_image = transforms.ToPILImage()(output_image)

# Save the output image
output_image.save(f"upscaled_{filename}")
